<img src="files/model_1_outline.png">

# change log
### Epoch = 800
### Shuffle=False


sequence generator shape (corrected) (b, n, w, h, c)

In [1]:
import tensorflow as tf
import datetime
import os
import glob
import keras

In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [5]:
classes = ['CalotTriangleDissection',
 'CleaningCoagulation',
 'ClippingCutting',
 'GallbladderDissection',
 'GallbladderPackaging',
 'GallbladderRetraction',
 'Preparation']


classes.sort()
print(classes)

['CalotTriangleDissection', 'CleaningCoagulation', 'ClippingCutting', 'GallbladderDissection', 'GallbladderPackaging', 'GallbladderRetraction', 'Preparation']


In [73]:
# some global params
SIZE = (224, 224) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [74]:
# pattern to get videos and classes
glob_pattern='../../cholec80/model_data/{classname}/*.avi'


In [75]:
# for data augmentation
# data_aug = keras.preprocessing.image.ImageDataGenerator(
#     zoom_range=.1,
#     horizontal_flip=True,
#     rotation_range=8,
#     width_shift_range=.2,
#     height_shift_range=.2)

In [76]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=False,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class CalotTriangleDissection, validation count: 170, train count: 347
class CleaningCoagulation, validation count: 170, train count: 347
class ClippingCutting, validation count: 170, train count: 347
class GallbladderDissection, validation count: 170, train count: 347
class GallbladderPackaging, validation count: 170, train count: 347
class GallbladderRetraction, validation count: 170, train count: 347
class Preparation, validation count: 170, train count: 347
Total data: 7 classes for 2429 files for train


In [77]:
# getting validation data
valid = train.get_validation_generator()

Total data: 7 classes for 1190 files for validation


In [78]:
import keras_video.utils
#keras_video.utils.show_sample(train)

In [12]:
#valid.files

## BUILD CONV NET

In [79]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
#from keras.applications.mobilenet import MobileNet
from keras.applications import ResNet50 
def build_mobilenet(shape=(224, 224, 3), nbout=3):
#     model = MobileNet(
#         include_top=False,
#         input_shape=shape,
#         weights='imagenet')
#     # Keep 9 layers to train﻿﻿
#     trainable = 9
#     for layer in model.layers[:-trainable]:
#         layer.trainable = False
#     for layer in model.layers[-trainable:]:
#         layer.trainable = True
    model = ResNet50(include_top=True, weights='imagenet')
    #return keras.Sequential([model])
    return Model(inputs=model.input, outputs=model.layers[-2].output)

## Build LSTM

In [80]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout, LSTM
from keras.initializers import GlorotNormal
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    model.add(Dense(512))
    # here, you can also use GRU or LSTM
    #model.add(LSTM(512, kernel_initializer='glorot_normal', return_sequences=False, input_shape=(2048, 5)))
    #model.add(LSTM(512, kernel_initializer='glorot_normal', return_sequences=False,  input_shape=(2048, 5, 8)))
    model.add(LSTM(512, kernel_initializer='glorot_normal'))
    # and finally, we make a decision network
    model.add(Dense(nbout, activation='linear'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [81]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 128, 128, 3)
print(INSHAPE)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

#this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

(5, 224, 224, 3)


## epochs, call backs

In [82]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/20210621-152703


In [83]:
EPOCHS=800
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_3_1_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/800
303/303 [==============================] - 200s 580ms/step - loss: 6.8461 - acc: 0.1423 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00001: saving model to model2_3_1_chkp\weights.01-6.94.hdf5
Epoch 2/800
303/303 [==============================] - 57s 188ms/step - loss: 7.1571 - acc: 0.1337 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00002: saving model to model2_3_1_chkp\weights.02-6.94.hdf5
Epoch 3/800
303/303 [==============================] - 57s 188ms/step - loss: 7.5666 - acc: 0.1449 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00003: saving model to model2_3_1_chkp\weights.03-6.94.hdf5
Epoch 4/800
303/303 [==============================] - 57s 188ms/step - loss: 6.8465 - acc: 0.1445 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00004: saving model to model2_3_1_chkp\weights.04-6.94.hdf5
Epoch 5/800
303/303 [==============================] - 57s 188ms/step - loss: 7.3350 - acc: 0.1442 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00005: saving model to model2_3_1_chkp\weigh

Epoch 40/800
303/303 [==============================] - 57s 187ms/step - loss: 6.2518 - acc: 0.1592 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00040: saving model to model2_3_1_chkp\weights.40-6.94.hdf5
Epoch 41/800
303/303 [==============================] - 57s 188ms/step - loss: 6.6759 - acc: 0.1114 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00041: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.

Epoch 00041: saving model to model2_3_1_chkp\weights.41-6.94.hdf5
Epoch 42/800
303/303 [==============================] - 57s 187ms/step - loss: 6.8398 - acc: 0.1488 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00042: saving model to model2_3_1_chkp\weights.42-6.94.hdf5
Epoch 43/800
303/303 [==============================] - 57s 187ms/step - loss: 6.9173 - acc: 0.1529 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00043: saving model to model2_3_1_chkp\weights.43-6.94.hdf5
Epoch 44/800
303/303 [==============================] - 57s 188ms/step - loss: 6.7683 - acc: 0.1461 - 


Epoch 00078: saving model to model2_3_1_chkp\weights.78-6.94.hdf5
Epoch 79/800
303/303 [==============================] - 57s 187ms/step - loss: 6.4564 - acc: 0.1572 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00079: saving model to model2_3_1_chkp\weights.79-6.94.hdf5
Epoch 80/800
303/303 [==============================] - 57s 187ms/step - loss: 5.8552 - acc: 0.1583 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00080: saving model to model2_3_1_chkp\weights.80-6.94.hdf5
Epoch 81/800
303/303 [==============================] - 57s 187ms/step - loss: 6.8881 - acc: 0.1360 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00081: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-11.

Epoch 00081: saving model to model2_3_1_chkp\weights.81-6.94.hdf5
Epoch 82/800
303/303 [==============================] - 57s 187ms/step - loss: 7.2643 - acc: 0.1630 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00082: saving model to model2_3_1_chkp\weights.82-6.94.hdf5
Epoch 83/800
303/303 [==============

303/303 [==============================] - 57s 188ms/step - loss: 7.5453 - acc: 0.1377 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00117: saving model to model2_3_1_chkp\weights.117-6.94.hdf5
Epoch 118/800
303/303 [==============================] - 57s 188ms/step - loss: 7.8861 - acc: 0.1219 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00118: saving model to model2_3_1_chkp\weights.118-6.94.hdf5
Epoch 119/800
303/303 [==============================] - 57s 187ms/step - loss: 6.5861 - acc: 0.1348 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00119: saving model to model2_3_1_chkp\weights.119-6.94.hdf5
Epoch 120/800
303/303 [==============================] - 57s 187ms/step - loss: 6.1983 - acc: 0.1383 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00120: saving model to model2_3_1_chkp\weights.120-6.94.hdf5
Epoch 121/800
303/303 [==============================] - 57s 187ms/step - loss: 6.7853 - acc: 0.1402 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00121: ReduceLROnPlateau reducing learning ra

303/303 [==============================] - 56s 186ms/step - loss: 7.1760 - acc: 0.1505 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00155: saving model to model2_3_1_chkp\weights.155-6.94.hdf5
Epoch 156/800
303/303 [==============================] - 56s 186ms/step - loss: 7.1175 - acc: 0.1543 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00156: saving model to model2_3_1_chkp\weights.156-6.94.hdf5
Epoch 157/800
303/303 [==============================] - 56s 186ms/step - loss: 6.2600 - acc: 0.1659 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00157: saving model to model2_3_1_chkp\weights.157-6.94.hdf5
Epoch 158/800
303/303 [==============================] - 56s 186ms/step - loss: 6.4018 - acc: 0.1411 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00158: saving model to model2_3_1_chkp\weights.158-6.94.hdf5
Epoch 159/800
303/303 [==============================] - 56s 186ms/step - loss: 6.8205 - acc: 0.1174 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00159: saving model to model2_3_1_chkp\weight

...

303/303 [==============================] - 57s 187ms/step - loss: 7.5333 - acc: 0.1603 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00308: saving model to model2_3_1_chkp\weights.308-6.94.hdf5
Epoch 309/800
303/303 [==============================] - 57s 187ms/step - loss: 7.0284 - acc: 0.1433 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00309: saving model to model2_3_1_chkp\weights.309-6.94.hdf5
Epoch 310/800
303/303 [==============================] - 57s 187ms/step - loss: 6.7138 - acc: 0.1278 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00310: saving model to model2_3_1_chkp\weights.310-6.94.hdf5
Epoch 311/800
303/303 [==============================] - 57s 187ms/step - loss: 6.4121 - acc: 0.1563 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00311: ReduceLROnPlateau reducing learning rate to 1.0000002074132203e-34.

Epoch 00311: saving model to model2_3_1_chkp\weights.311-6.94.hdf5
Epoch 312/800
303/303 [==============================] - 56s 186ms/step - loss: 6.7697 - acc: 0.1192 - val_l

303/303 [==============================] - 57s 187ms/step - loss: 7.4246 - acc: 0.1298 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00346: saving model to model2_3_1_chkp\weights.346-6.94.hdf5
Epoch 347/800
303/303 [==============================] - 57s 187ms/step - loss: 7.6036 - acc: 0.1082 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00347: saving model to model2_3_1_chkp\weights.347-6.94.hdf5
Epoch 348/800
303/303 [==============================] - 57s 187ms/step - loss: 6.9694 - acc: 0.1528 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00348: saving model to model2_3_1_chkp\weights.348-6.94.hdf5
Epoch 349/800
303/303 [==============================] - 57s 187ms/step - loss: 7.5048 - acc: 0.1220 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00349: saving model to model2_3_1_chkp\weights.349-6.94.hdf5
Epoch 350/800
303/303 [==============================] - 57s 187ms/step - loss: 6.9255 - acc: 0.1748 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00350: saving model to model2_3_1_chkp\weight

303/303 [==============================] - 56s 186ms/step - loss: 7.0565 - acc: 0.1207 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00384: saving model to model2_3_1_chkp\weights.384-6.94.hdf5
Epoch 385/800
303/303 [==============================] - 56s 186ms/step - loss: 6.1737 - acc: 0.1663 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00385: saving model to model2_3_1_chkp\weights.385-6.94.hdf5
Epoch 386/800
303/303 [==============================] - 57s 187ms/step - loss: 7.1291 - acc: 0.1710 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00386: saving model to model2_3_1_chkp\weights.386-6.94.hdf5
Epoch 387/800
303/303 [==============================] - 56s 186ms/step - loss: 7.3128 - acc: 0.1223 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00387: saving model to model2_3_1_chkp\weights.387-6.94.hdf5
Epoch 388/800
303/303 [==============================] - 56s 186ms/step - loss: 6.6507 - acc: 0.1346 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00388: saving model to model2_3_1_chkp\weight

303/303 [==============================] - 56s 186ms/step - loss: 6.7735 - acc: 0.1726 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00422: saving model to model2_3_1_chkp\weights.422-6.94.hdf5
Epoch 423/800
303/303 [==============================] - 57s 187ms/step - loss: 7.1431 - acc: 0.1352 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00423: saving model to model2_3_1_chkp\weights.423-6.94.hdf5
Epoch 424/800
303/303 [==============================] - 57s 188ms/step - loss: 6.7497 - acc: 0.1505 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00424: saving model to model2_3_1_chkp\weights.424-6.94.hdf5
Epoch 425/800
303/303 [==============================] - 57s 187ms/step - loss: 7.4656 - acc: 0.1289 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00425: saving model to model2_3_1_chkp\weights.425-6.94.hdf5
Epoch 426/800
303/303 [==============================] - 56s 186ms/step - loss: 6.4749 - acc: 0.1562 - val_loss: 6.9428 - val_acc: 0.1436

Epoch 00426: saving model to model2_3_1_chkp\weight

KeyboardInterrupt: 

In [ ]:
LSTM(512, kernel_initializer='glorot_normal', input_shape = (1,2,3))